In [1]:
import torch
from torch_geometric.data import Data

# data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

# data.x: Node feature matrix with shape [num_nodes, num_node_features]
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

# data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
# data.y: Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]
# data.pos: Node position matrix with shape [num_nodes, num_dimensions]

data = Data(x=x, edge_index=edge_index)
print(data)
print(data.keys)
print(data['x'])
print(data['edge_index'])

Data(edge_index=[2, 4], x=[3, 1])
['x', 'edge_index']
tensor([[-1.],
        [ 0.],
        [ 1.]])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [2]:
### Data Loading
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

for batch in loader:
    print(batch)

600
6
21
Batch(batch=[1002], edge_index=[2, 3774], x=[1002, 21], y=[32])
Batch(batch=[1114], edge_index=[2, 4298], x=[1114, 21], y=[32])
Batch(batch=[1271], edge_index=[2, 4454], x=[1271, 21], y=[32])
Batch(batch=[890], edge_index=[2, 3396], x=[890, 21], y=[32])
Batch(batch=[1089], edge_index=[2, 3766], x=[1089, 21], y=[32])
Batch(batch=[1014], edge_index=[2, 3966], x=[1014, 21], y=[32])
Batch(batch=[1101], edge_index=[2, 4248], x=[1101, 21], y=[32])
Batch(batch=[1049], edge_index=[2, 4140], x=[1049, 21], y=[32])
Batch(batch=[979], edge_index=[2, 3840], x=[979, 21], y=[32])
Batch(batch=[1053], edge_index=[2, 3864], x=[1053, 21], y=[32])
Batch(batch=[1053], edge_index=[2, 4266], x=[1053, 21], y=[32])
Batch(batch=[1139], edge_index=[2, 4346], x=[1139, 21], y=[32])
Batch(batch=[1030], edge_index=[2, 4084], x=[1030, 21], y=[32])
Batch(batch=[1002], edge_index=[2, 3940], x=[1002, 21], y=[32])
Batch(batch=[848], edge_index=[2, 3208], x=[848, 21], y=[32])
Batch(batch=[1012], edge_index=[2, 39

In [91]:
### Example of GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data

dataset = Planetoid(root='/tmp/Cora', name='Cora')

dataset_quantized = Data(x=dataset[0].x.type(torch.int32), edge_index=dataset[0].edge_index, y=dataset[0].y, val_mask=dataset[0].val_mask, test_mask=dataset[0].test_mask, train_mask=dataset[0].train_mask)

print(dataset.__dict__)
print(dataset[0].__dict__)
print(dataset_quantized.__dict__)


{'name': 'Cora', 'root': '/tmp/Cora', 'transform': None, 'pre_transform': None, 'pre_filter': None, '__indices__': None, 'data': Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708]), 'slices': {'x': tensor([   0, 2708]), 'edge_index': tensor([    0, 10556]), 'y': tensor([   0, 2708]), 'train_mask': tensor([   0, 2708]), 'val_mask': tensor([   0, 2708]), 'test_mask': tensor([   0, 2708])}, '__data_list__': [Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])], 'split': 'public'}
{'x': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'edge_index': tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]]), 'edge_attr': None, 'y': t

In [107]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Parameter

### Non-quantized
# Training
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16) # 1433->16
        self.conv2 = GCNConv(16, dataset.num_classes)       # 16->7

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

### quantized
# Training
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16) # 1433->16
        self.conv2 = GCNConv(16, dataset.num_classes)       # 16->7
        self.conv1.weight = Parameter(torch.Tensor(dataset.num_node_features, 16).type(torch.int32))
        self.conv1.bias = Parameter(torch.Tensor(16, dtype=torch.int32))
        self.conv2.weight = Parameter(torch.Tensor(16, dataset.num_classes, dtype=torch.int32))
        self.conv2.bias = Parameter(torch.Tensor(dataset.num_classes, dtype=torch.int32))
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset_quantized.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7690


RuntimeError: Only Tensors of floating point and complex dtype can require gradients

In [123]:
# Self implementing GCNConv using Message Passing
try:# GCNConv is imported in previous cell, here we're defining our own
    if GCNConv:
        print("deleting GCNConv")
        del GCNConv 
except NameError:
    pass


import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class myGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(myGCNConv, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

deleting GCNConv


In [124]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = myGCNConv(dataset.num_node_features, 16) # 1433->16
        self.conv2 = myGCNConv(16, dataset.num_classes)       # 16->7

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
# Evaluation
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8170


In [187]:
from __future__ import division

from collections import defaultdict
from functools import lru_cache, reduce
import math
import operator

import torch


def profile(module, display_cpu=True, display_gpu=True):
    assert issubclass(module, torch.nn.Module)
    monkey_patch_init(module)
    return module

def monkey_patch_init(_class):
    old_init = _class.__init__
    def new_init(self, *args, **kwargs):
        old_init(self, *args, **kwargs)
        self.profiler = Profiler(self)
        _class.__str__ = self.profiler.__str__
    _class.__init__ = new_init

class Profiler(object):
    def __init__(self, module):
        """
        An operation is a graph node that performs computation on tensors.
        """
        self._module = module
        self._events = {
            'forward': defaultdict(Event),
            'backward': defaultdict(Event)}
        self._operations = {}
        self._enable = True

        #consume generator
        list(map(self._hook_operation, operations(self._module)))
    
    def _hook_operation(self, op):
        def wrapper_call(op, *input, **kwargs):
            # Wrapper function to "__call__", with time counter in it.
            if not self._enable:
                return self._operations[op.__class__](op, *input, **kwargs)

            with torch.autograd.profiler.profile() as prof:
                result = self._operations[op.__class__](op, *input, **kwargs)
            
            self._events['forward'][op] += Event(
                cpu_time=int(prof.total_average().cpu_time),
                gpu_time=int(prof.total_average().cuda_time),
                parameters=count_elements(op.parameters()),
                input_size=count_elements(input),
                hits=1)
            
            def backward_pre_hook(*args):
                if not self._enable:
                    return
                self._events['backward'][op].append(time.time())
            #result.grad_fn.register_pre_hook(backward_pre_hook);
            return result

        # monky patch "__call__" with "wrapper_call"  for this operation`
        if op.__class__ not in self._operations:
            self._operations[op.__class__] = op.__class__.__call__
            op.__class__.__call__ = wrapper_call

        #def backward_post_hook(*args):
        #    if not this_profiler.profiling_on:
        #        return
        #    # adds ending time
        #    backward = this_profiler.record['backward']
        #    backward[-1] = backward[-1] + (time.time(),) 
        #op.register_backward_hook(backward_post_hook)   
    
    @lru_cache(maxsize=None)
    def get_metrics(self, module):
        if module in self._events['forward']:
            #it's an operation
            return self._events['forward'][module]
        
        return reduce(operator.add, map(self.get_metrics, module._modules.values()))
    
    def __str__(self, module=None, indentation=0, pre_msg=''):
        tmpstr = ''
        if module is None:
            module = self._module
            tmpstr += Event.header()

        # this is an operation
        metrics = self.get_metrics(module).tostring()
    
        if module.__class__ in self._operations:
            return  tmpstr + metrics + indent(pre_msg + module.__repr__(), indentation) + '\n'
        
        name = module.__class__.__name__
        tmpstr += metrics + indent(pre_msg + name  + '(', indentation) + '\n'
        for key, sub_module in module._modules.items():
            tmpstr +=  self.__str__(sub_module, indentation+2, pre_msg='(' + key + '): ')
        tmpstr +=  indent(')',indentation+len(metrics)) + '\n'
        return tmpstr        

class Event(object):
    def __init__(self, cpu_time=0, gpu_time=0, parameters=0, input_size=0, hits=0):
        self.cpu_time = cpu_time
        self.gpu_time = gpu_time
        self.parameters = parameters
        self.input_size = input_size
        self.hits = hits
    
    @classmethod
    def header(cls):
        header = format_columns(['CPU Time','GPU Time','Parameters','Input','Architecture'])
        return '\n'.join([header,'='*len(header),''])

    def tostring(self):
        return format_columns([
                format_time(self.cpu_time),
                format_time(self.gpu_time),
                format_count(self.parameters),
                format_count(self.input_size)])
    
    def __add__(self, other):
        return Event(
            self.cpu_time + other.cpu_time,
            self.gpu_time + other.gpu_time,
            self.parameters + other.parameters,
            self.input_size + other.input_size,
            self.hits + other.hits)

    def __radd__(self, other):
        return self.__add__(other)

def format_columns(cols, width=10):
    assert isinstance(cols, list)
    return  ' ' + ' '.join(col.center(width,' ') for col in cols) + '  '

def format_time(time_in_ns):
    if not time_in_ns:
        return '-'

    human_powers = ['n','u','m','']
    power = int(math.log(time_in_ns, 10) // 3)
    return '{:.2f}{}s '.format(
            time_in_ns/1000.**power,
            human_powers[power])

def format_count(n):
    if not n:
        return '-'

    human_powers = ['','k','m','g']
    power = int(math.log(n, 10) // 3)
    return '{:.2f}{} '.format(
            n/1000.**power,
            human_powers[power])

def operations(module):
    """
    Given a module recursively transverse it
    to find all atomic operations.

    Atomic operations are the nodes in the graph which
    perform computations on the tensors.
    """
    if not len(list(module.children())):
        # nn.Module who doesn't have sub nn.Module, hook it.
        yield module

    for name, sub_module in module.named_children():
        if (isinstance(sub_module, torch.nn.Container)
            or isinstance(sub_module, torch.nn.Sequential)
            or isinstance(sub_module, torch.nn.ModuleList)
            or isinstance(sub_module, torch.nn.Module)):
            # Recursively visit their decendants.
            for op in operations(sub_module): #python2 compatibility
                yield op

def indent(s, indent):
    return '\n'.join((indent* ' ') + line for line in s.split('\n'))

def count_elements(tensors):
    return sum([reduce(operator.mul, t.size()) for t in tensors])

In [23]:
# test SpMM
# ! rm -rf Planetoid Planetoid_toTensor
import torch
import torch.nn.functional as F
import time
import numpy as np

from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

from torchsummary import summary

# without SpMM
dataset1 = Planetoid("Planetoid", name="Cora")
data1 = dataset1[0]
# print(data1)

class Net1(torch.nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = GCNConv(dataset1.num_features, 16, cached=True) # 1433->16
        self.conv2 = GCNConv(16, dataset1.num_classes, cached=True)  # 16->7

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        start = time.time()
        x = self.conv1(x, edge_index)
        end = time.time()
        print("Conv1 time:", end-start)
        start = time.time()
        x = F.relu(x)
        end = time.time()
        print("Relu time:", end-start)
        start = time.time()
        x = self.conv2(x, edge_index)
        end = time.time()
        print("Conv2 time:", end-start)
        return F.log_softmax(x, dim=1)

# Training
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net1()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(data):
#     with torch.autograd.profiler.profile() as prof:
    model.train()
#     print(prof.key_averages().table(sort_by="self_cpu_time_total"))
    optimizer.zero_grad()
    out = model(data)
#     print(out)
#     print(out.shape)X
#     print(data.x)
#     print("Density: ", np.count_nonzero(data.x.cpu().detach().numpy())/data.x.shape[0]/data.x.shape[1])
#     print("Density: ", np.count_nonzero(out.cpu().detach().numpy())/out.shape[0]/out.shape[1])
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    return float(loss)
    
start = time.time()
for epoch in range(1):
    for name, param in model.named_parameters():
        if "weight" in name:
            print(f"Epoch {epoch}:", name, "Density:", np.count_nonzero(param.data.cpu().detach().numpy())/param.data.shape[0]/param.data.shape[1])
    loss = train(data1)
#     print(data1.x.shape)
#     print(data1.x)
#     print(np.count_nonzero(data1.x))
end = time.time()
print(f"Time without SpMM: {end-start}")
print("")

Epoch 0: conv1.weight Density: 1.0
Epoch 0: conv2.weight Density: 1.0
Conv1 time: 0.0061147212982177734
Relu time: 0.00012755393981933594
Conv2 time: 0.0020704269409179688
Time without SpMM: 0.01825571060180664



In [ ]:
# ------------------------------------------------------ #

if False:
    # with SpMM
    dataset2 = Planetoid("Planetoid_toTensor", name="Cora", transform=T.ToSparseTensor())
    data2 = dataset2[0]


    class Net2(torch.nn.Module):
        def __init__(self):
            super(Net2, self).__init__()
            self.conv1 = GCNConv(dataset2.num_features, 16, cached=True)
            self.conv2 = GCNConv(16, dataset2.num_classes, cached=True)

        def forward(self, x, adj_t):
            x = self.conv1(x, adj_t)
            x = F.relu(x)
            x = self.conv2(x, adj_t)
            return F.log_softmax(x, dim=1)

    model = Net2()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    def train(data):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.adj_t)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        return float(loss)


    start = time.time()
    for epoch in range(1, 201):
        loss = train(data2)
    end = time.time()
    print(f"Time with SpMM: {end-start}")
    print("") 